## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [2]:
import os
path = '../script'

# 다큐멘터리(0), 드라마(1), 영화(2) 선택
category = 2
if category == 0:
    path = path + '/Documentary'
elif category == 1:
    path = path + '/Drama'
else:
    path = path + '/Movie'

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

print(len(file_list), file_list[:5])

46 ['The.Intern', 'The.Departed', 'Ben.Platt.Live.from.Radio.City.Music.HallBen.Platt.Live.from.Radio.City.Music.Hall', "It's.Complicated", 'Baby.DriverBaby.Driver']


## 여기서 작품 선택합니다 꼭!!

In [3]:
# 바로 위 코드에서 file_list를 참고하여 작품 선택
title = file_list[0]

file_name = title + '.WEBRip.Netflix.en[cc]'
print(f'{path}/{title}/{file_name}')

../script/Movie/The.Intern/The.Intern.WEBRip.Netflix.en[cc]


## VTT 파일 읽기

In [4]:
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [5]:
segments[:5]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2016/07/07 23:49:41',
 'NOTE SegmentIndex\nNOTE Segment=596.846 29612@689 218\nNOTE Segment=594.511 32178@30301 248\nNOTE Segment=597.304 29740@62479 229\nNOTE Segment=595.888 28674@92219 221\nNOTE Segment=596.804 31138@120893 239\nNOTE Segment=598.055 27450@152031 213\nNOTE Segment=597.848 35970@179481 270\nNOTE Segment=598.264 36784@215451 279\nNOTE Segment=596.972 23038@252235 178\nNOTE Segment=597.471 30042@275273 231\nNOTE Segment=597.264 20117@305315 157\nNOTE Segment=413.413 15008@325432 118\nNOTE /SegmentIndex',
 '                                                                                          ',
 '                    \n1\n00:01:09.569 --> 00:01:12.447  position:50.00%,middle  align:middle size:80.00%  line:10.00% \nBEN: <i>Freud said,</i>\n<i>"Love and work. Work and love.</i>']

## 불필요한 부분 제거

In [6]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [7]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:5]

['                    \n1\n00:01:09.569  00:01:12.447  BEN: Freud said,\n"Love and work. Work and love.',
 '2\n00:01:12.614  00:01:14.324  "That\'s all there is."',
 "3\n00:01:14.491  00:01:17.327  Well, I'm retired, and my wife is dead.",
 "4\n00:01:17.494  00:01:18.745  As you can imagine, that's given me",
 '5\n00:01:18.911  00:01:20.663  some time on my hands.']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [10]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' '.join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [12]:
trimmed_segments[:5]

[['1',
  '00:01:09.569',
  '00:01:12.447',
  'BEN: Freud said, "Love and work. Work and love.'],
 ['2', '00:01:12.614', '00:01:14.324', '"That\'s all there is."'],
 ['3',
  '00:01:14.491',
  '00:01:17.327',
  "Well, I'm retired, and my wife is dead."],
 ['4', '00:01:17.494', '00:01:18.745', "As you can imagine, that's given me"],
 ['5', '00:01:18.911', '00:01:20.663', 'some time on my hands.']]

## 데이터 저장

In [13]:
with open(f'{path}/{title}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [54]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data.head()

,num,start,end,script
0,1,00:01:09.569,00:01:12.447,"BEN: Freud said, ""Love and work. Work and love."
1,2,00:01:12.614,00:01:14.324,"""That's all there is."""
2,3,00:01:14.491,00:01:17.327,"Well, I'm retired, and my wife is dead."
3,4,00:01:17.494,00:01:18.745,"As you can imagine, that's given me"
4,5,00:01:18.911,00:01:20.663,some time on my hands.


In [55]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
  if current_row['script'][-1] in ['.', '!', '?']:
    continue
  else:
    row_index = index + 1
    while True:
      next_row = data.loc[row_index]
      new_script = data.loc[index, 'script'] + ' ' + next_row['script']
      data.loc[index, 'script'] = new_script
      if next_row['script'][-1] in ['.', '!', '?']:
        data.loc[index, 'end'] = next_row['end']
        break
      else:
        row_index += 1

# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

,num,start,end,script
0,1,00:01:09.569,00:01:12.447,"BEN: Freud said, Love and work. Work and love."
1,2,00:01:12.614,00:01:14.324,That's all there is.
2,3,00:01:14.491,00:01:17.327,"Well, I'm retired, and my wife is dead."
3,4,00:01:17.494,00:01:20.663,"As you can imagine, that's given me some time on my hands."
5,6,00:01:21.164,00:01:23.416,My wife's been gone for three and a half years.
6,7,00:01:23.583,00:01:24.959,I miss her in every way.
7,8,00:01:25.127,00:01:28.838,"And retirement? That is an ongoing, relentless effort in creativity."
9,10,00:01:29.005,00:01:31.758,"At first, I admit I enjoyed the novelty of it."
10,11,00:01:31.924,00:01:33.301,Sort of felt like I was playing hooky.
11,12,00:01:33.926,00:01:37.013,I used all the miles I'd saved and traveled the globe.


In [56]:
data.to_csv(f'{path}/{title}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv')

data.head()

,start,end,script
0,00:01:09.569,00:01:12.447,"BEN: Freud said, Love and work. Work and love."
1,00:01:12.614,00:01:14.324,That's all there is.
2,00:01:14.491,00:01:17.327,"Well, I'm retired, and my wife is dead."
3,00:01:17.494,00:01:20.663,"As you can imagine, that's given me some time on my hands."
4,00:01:21.164,00:01:23.416,My wife's been gone for three and a half years.
